# Nenana Ice Classic Modeling
This notebook was used for modeling the NIC data.

In [1]:
# imports

import numpy as np
import pandas as pd

# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from mlxtend.plotting import plot_decision_regions

# pipeline and preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from category_encoders import WOEEncoder, OneHotEncoder
from sklearn.decomposition import KernelPCA as KPCA
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import ColumnSelector

# models
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from mlxtend.classifier import StackingCVClassifier

# evaluation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, cross_validate, cross_val_predict
from sklearn import metrics # explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
#from sklearn.utils import check_arrays # converts list-like objects to array (if needed)

# import datetime # didn't need after all

# Pretty Print to make some outputs easier to read
import pprint

# filter warnings
import warnings
warnings.filterwarnings("ignore")

import gc # garbage collection

rng = np.random.RandomState(42)

In [2]:
def stringify(data):
    data = pd.DataFrame(data)
    for c in data.columns.tolist():
        data[c] = data[c].astype(str)
    return data

### Collapsed Cells
The cells collapsed below were used to create the 'features_added.csv' data file that is loaded later. They are collapsed because they are commented out and not run. Instead, the file is read into a DataFrame.

In [3]:
# Bryan suggested rolling avg, rolling slope, rolling acceleration, and rolling std dev features might help
# improve model performance

# def slope(y):
#     x = range(len(y))
#     # y = mx + b
#     m, b = np.polyfit(x, y, 1)
#     return m

# def accel(y):
#     t = range(len(y))
#     # y = 1/2 g t^2 + v t + y0
#     a, v, y0 = np.polyfit(t, y, 2)
#     return 5*a

In [4]:
# # read data
# df = pd.read_csv('../data/cleaned_data.csv')

In [5]:
# df.head()

In [6]:
# df.drop(columns = 'Unnamed: 0', inplace = True)

In [7]:
# df.shape

## Feature Engineering

In [8]:
# df.info()

In [9]:
# df['Date'] = pd.to_datetime(df['Date'])

In [10]:
# df['winningTime'] = pd.to_datetime(df['winningTime'], errors = 'ignore')

In [11]:
# create column for ordinal day of year
# df['dayOfYear'] = df['Date'].dt.dayofyear

In [12]:
# # create binary 'winningDate' column
# df['winningDate'] = 0
# idx = df.loc[df['winningTime'] != '0'].index
# df['winningDate'].loc[idx] = 1

In [13]:
# df.loc[df['winningDate'] == 1]

The information for 1995 and 1999 are missing some dates, including the winning date. I decided to drop the data for those years, since there is no target as a result. Survival analysis would also falsely treat those years as censored.

In [14]:
# drop_1995_1999 = df.loc[(df['Date'].dt.year == 1995)|(df['Date'].dt.year == 1999)].index

In [15]:
# df.drop(index = drop_1995_1999, inplace = True)

Drop all records for a year that are after the ice broke.

In [16]:
# year_list = sorted(list(set(df['Date'].dt.year)))

In [17]:
# pprint.pprint(year_list, compact = True)

In [18]:
# get winning date indices
# idx_w = df.loc[df['winningDate'] == 1].index
# idx_w

In [19]:
# make a list of indices to drop
# drop_index = []
# for i, year in enumerate(year_list):
#     idx_y = df.loc[df['Date'].dt.year == year].index
#     for idx in idx_y:
#         if idx > idx_w[i]:
#             drop_index.append(idx)
#         else:
#             pass

In [20]:
# # drop observations that occurred after the winning date in a year
# for idx in drop_index:
#     df.drop(index = idx, inplace = True)

In [21]:
# df.shape

In [22]:
# df.info()

In [23]:
# df['past'] = (df['Date'] < '2015-01-01').astype(np.int)

In [ ]:
# df['future'] = 1 - df['past']

In [ ]:
# df.head(3).append(df.tail(3))

In [ ]:
# df['precipType'].value_counts()

In [ ]:
# # encode precipType
# df = df.merge(pd.get_dummies(df['precipType'], prefix = 'precip', drop_first = True, sparse = True),
#          how = 'left', left_index = True, right_index = True)

In [ ]:
# # drop precipType after encoding
# df.drop(columns = 'precipType', inplace = True)
# gc.collect()

Create column for daily average temperature

In [ ]:
# df['temperatureAvg'] = (df['temperatureMin'] + df['temperatureMax']) / 2

In [ ]:
# plots showing examples of additional time-series features mentioned above; kept for example/reminder
# temp = df['temperatureAvg'].head(90).copy()

# temp.plot()
# temp.rolling(5).mean().plot()
# plt.show()

# temp.rolling(5).apply(lambda x: slope(x)).plot()
# temp.rolling(10).apply(lambda x: accel(x)).plot()
# temp.rolling(10).std().plot()
# plt.show()

Create columns for number of "hot days," "cold days," and snow accumulated since Apr 1 in a given year.

I defined a "hot day" as a day where: day_average_temp > median(year_avg_temp) + std_dev(year_avg_temp)

A "cold day" is a day where: day_average_temp < median(year_avg_temp) - std_dev(year_avg_temp)

In [ ]:
# hot_count = []
# cold_count = []
# daily_accumulation = []
# for year in year_list:
#     hot_temp_count = 0
#     cold_temp_count = 0
#     daily_accum = 0
#     temp_df = df.loc[df['Date'].dt.year == year]
#     hot_threshold = temp_df['temperatureAvg'].median() + temp_df['temperatureAvg'].std()
#     cold_threshold = temp_df['temperatureAvg'].median() - temp_df['temperatureAvg'].std()
#     for idx in temp_df.index:
#         current_temp = temp_df['temperatureAvg'].loc[idx]
#         if temp_df['precip_snow'].loc[idx] == 1:
#             daily_accum += temp_df['precipAccumulation'].loc[idx]
#         else:
#             pass
#         if current_temp >= hot_threshold:
#             hot_temp_count += 1
#         elif current_temp <= cold_threshold:
#             cold_temp_count += 1
#         else:
#             pass
#         hot_count.append(hot_temp_count)
#         cold_count.append(cold_temp_count)
#         daily_accumulation.append(daily_accum)

In [ ]:
# df['numHotDays'] = hot_count
# df['numColdDays'] = cold_count
# df['accumulatedSnow'] = daily_accumulation

#### Save data before adding rolling average features.

In [ ]:
# df.to_csv('../data/pre-moving-average_data.csv', index = False)

Create columns for moving average features.

In [ ]:
# ma_cols = ['humidity', 'windSpeed', 'windBearing', 'uvIndex', 'precipIntensity', 'iceThickness', 'temperatureAvg', 'numHotDays', 'numColdDays']
# windows = [3, 5, 7, 10]

In [ ]:
# # first add new columns with dummy info
# for col in ma_cols:
#     for window in windows:
#         label_ma = col + '_MA' + str(window)
#         df[label_ma] = 0
        
#         label_slope = col + '_MA-slope' + str(window)
#         df[label_slope] = 0
        
#         label_accel = col + '_MA-accel' + str(window)
#         df[label_accel] = 0
        
#         label_std = col + '_MA-std_dev' + str(window)
#         df[label_std] = 0

In [ ]:
# # Update each year with its rolling averages
# %time
# for year in year_list:
#     temp_df = df.loc[df['Date'].dt.year == year]
#     for col in ma_cols:
#         for window in windows:
#             # assign labels
#             label_ma = col + '_MA' + str(window)
#             label_slope = col + '_MA-slope' + str(window)
#             label_accel = col + '_MA-accel' + str(window)
#             label_std = col + '_MA-std_dev' + str(window)
#             # for each year, update row values in new columns
#             for idx in temp_df.index:
#                 df[label_ma].loc[idx] = temp_df[col].rolling(window).mean().loc[idx]
#                 df[label_slope].loc[idx] = temp_df[col].rolling(window).apply(lambda x: slope(x)).loc[idx]
#                 df[label_accel].loc[idx] = temp_df[col].rolling(window).apply(lambda x: accel(x)).loc[idx]
#                 df[label_std].loc[idx] = temp_df[col].rolling(window).std().loc[idx]

In [ ]:
# df.sample(7)

In [ ]:
# df.shape

#### Save results to file

In [ ]:
# df.to_csv('../data/features_added.csv', index = False)

#### Read results from file

In [3]:
df = pd.read_csv('../data/features_added.csv')

In [4]:
df.head()

,Date,moonPhase,humidity,windSpeed,windBearing,uvIndex,temperatureMin,temperatureMax,precipIntensity,precipAccumulation,...,numColdDays_MA-accel5,numColdDays_MA-std_dev5,numColdDays_MA7,numColdDays_MA-slope7,numColdDays_MA-accel7,numColdDays_MA-std_dev7,numColdDays_MA10,numColdDays_MA-slope10,numColdDays_MA-accel10,numColdDays_MA-std_dev10
0,1989-03-01,0.80,0.70,8.42,236.0,1.0,20.58,29.65,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1989-03-02,0.83,0.68,8.59,266.0,1.0,-7.38,29.07,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1989-03-03,0.87,0.50,5.84,344.0,1.0,-19.23,1.84,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1989-03-04,0.90,0.51,2.52,6.0,1.0,-30.34,4.69,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1989-03-05,0.94,0.56,1.76,216.0,1.0,-38.53,0.74,0.0,0.0,...,0.714286,1.30384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Modeling

#### How good does my model have to be?

In [5]:
# the thing to beat: 0.9827072152653548
print('Percentage of non-events:')
1 - (df['winningDate'].sum()/df['winningDate'].count())

Percentage of non-events:


0.9827072152653548

#### Drop columns that are highly correlated
* temperatureMin and temperatureMax information was captured in temperatureAvg
* precipAccumulation information was captured in accumulatedSnow

In [6]:
df.drop(columns = ['temperatureMin', 'temperatureMax', 'precipAccumulation'],
        inplace = True)

#### Create training and testing DataFrames

In [7]:
train = df.loc[df['past'] == 1]
train.drop(columns = ['past', 'future'], inplace = True)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1399 entries, 0 to 1398
Columns: 162 entries, Date to numColdDays_MA-std_dev10
dtypes: float64(154), int64(6), object(2)
memory usage: 1.7+ MB


In [9]:
test = df.loc[df['future'] == 1]
test.drop(columns = ['past', 'future'], inplace = True)

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 1399 to 1676
Columns: 162 entries, Date to numColdDays_MA-std_dev10
dtypes: float64(154), int64(6), object(2)
memory usage: 354.0+ KB


In [11]:
df.drop(columns = ['past', 'future'], inplace = True)
gc.collect()

11

#### Create a DataFrame to track model performance
Eventually I'm going to get around to automating scoring tracking...

In [ ]:
# scoring_tracker = pd.DataFrame(columns = ['Model', 'Accuracy', 'ROC_AUC',
#                                           'F1', 'F1 (weighted)',
#                                           'Precision', 'Precision (weighted)',
#                                           'Recall', 'Recall (weighted)'])

In [12]:
# columns to exclude from models (dates and target info)
exclude = ['winningTime', 'winningDate',  'Date']

In [13]:
used_cols = [c for c in train.columns if c not in exclude]

In [14]:
no_ma_cols = [c for c in df.columns.tolist() if not c.endswith(('3', '5', '7', '10'))]
used_no_ma_cols = [c for c in no_ma_cols if c not in exclude]

In [15]:
pprint.pprint(no_ma_cols)
pprint.pprint(used_no_ma_cols)

['Date',
 'moonPhase',
 'humidity',
 'windSpeed',
 'windBearing',
 'uvIndex',
 'precipIntensity',
 'winningTime',
 'daylightHours',
 'iceThickness',
 'dayOfYear',
 'winningDate',
 'precip_rain',
 'precip_snow',
 'temperatureAvg',
 'numHotDays',
 'numColdDays',
 'accumulatedSnow']
['moonPhase',
 'humidity',
 'windSpeed',
 'windBearing',
 'uvIndex',
 'precipIntensity',
 'daylightHours',
 'iceThickness',
 'dayOfYear',
 'precip_rain',
 'precip_snow',
 'temperatureAvg',
 'numHotDays',
 'numColdDays',
 'accumulatedSnow']


In [16]:
train.dropna(inplace = True)

In [17]:
test.dropna(inplace = True)

In [18]:
gc.collect()

0

_Notes/Ideas_
* ~~add feature for day-of-year (to make the models time-aware);~~
* try hidden Markov model;
* ~~look at survival analysis/time-to-event analysis;~~
* look for outliers in training data, if removed, does model performance improve?;
* is there any way to identify mechanical failure of ice vs mush-out?;
* more time-series-like features, for instance number of "hot" days vs number of "cold" days; lags and rolling averages that Bryan talked about
* consider under- or over-sampling
* try random forest model;
* try knn model;